In [3]:
import os,requests,json;
from ipywidgets import IntProgress,HTML,VBox;
from IPython.display import display;

r = requests.get(
    'http://dz_gs:8080/geoserver/rest/settings/contact'
   ,auth=('admin','nhdplus')
);

if r.status_code != 200 or r.json()["contact"]["contactOrganization"] != 'NHDPlusInABox':
   raise Exception('geoserver does not appear ready for configuration');

r = requests.get(
    'http://dz_gs:8080/geoserver/rest/workspaces'
   ,auth=('admin','nhdplus')
);

boo_check = False;
for item in r.json()["workspaces"]["workspace"]:
   if item["name"] == "nhdplus":
      boo_check = True;
if not boo_check:
   r = requests.post(
       'http://dz_gs:8080/geoserver/rest/workspaces'
      ,headers={'Content-Type':'application/json'}
      ,params={'default':True}
      ,data=json.dumps({'workspace':{'name':'nhdplus'}})
      ,auth=('admin','nhdplus')
   );
   
r = requests.get(
    'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/datastores'
   ,auth=('admin','nhdplus')
);
if r.status_code != 200:
   raise Exception('datastores get failed');

boo_check = False;
if r.json()["dataStores"] != "":
   for item in r.json()["dataStores"]["dataStore"]:
      if item["name"] == "dzpg_nhdplus":
         boo_check = True;
if not boo_check:
   payload = {
      "dataStore": {
          "name": "dzpg_nhdplus"
         ,"connectionParameters": {
            "entry": [
                {"@key":"host"    ,"$":"dz_pg"}
               ,{"@key":"port"    ,"$":"5432"}
               ,{"@key":"database","$":"nhdplus"}
               ,{"@key":"user"    ,"$":"nhdplus"}
               ,{"@key":"passwd"  ,"$":os.environ['POSTGRES_PASSWORD']}
               ,{"@key":"dbtype"  ,"$":"postgis"}
               ,{"@key":"schema"  ,"$":"nhdplus"}
            ]
         }
      }
   }
   r = requests.post(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/datastores'
      ,headers={'Content-Type':'application/json'}
      ,data=json.dumps(payload)
      ,auth=('admin','nhdplus')
   );

r = requests.get(
    'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/styles'
   ,auth=('admin','nhdplus')
);   
if r.status_code != 200:
   raise Exception('styles get failed');
   
sty = [];
if r.json()["styles"] != "":
   for item in r.json()["styles"]["style"]:
      sty.append(item["name"]);
   
if 'catchment_polygon' not in sty:
   payload = """<?xml version="1.0" encoding="UTF-8"?>
<StyledLayerDescriptor version="1.0.0" 
 xsi:schemaLocation="http://www.opengis.net/sld StyledLayerDescriptor.xsd" 
 xmlns="http://www.opengis.net/sld" 
 xmlns:ogc="http://www.opengis.net/ogc" 
 xmlns:xlink="http://www.w3.org/1999/xlink" 
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
  <NamedLayer>
    <Name>catchment_polygon</Name>
    <UserStyle>
      <FeatureTypeStyle>
        <Rule>
          <Name>Viewable</Name>
          <MaxScaleDenominator>288896</MaxScaleDenominator>
          <PolygonSymbolizer>
            <Fill>
              <CssParameter name="fill">#AAAAAA</CssParameter>
              <CssParameter name="fill-opacity">0</CssParameter>
            </Fill>
            <Stroke>
              <CssParameter name="stroke">#E67000</CssParameter>
              <CssParameter name="stroke-width">1.5</CssParameter>
            </Stroke>
          </PolygonSymbolizer>
        </Rule>
      </FeatureTypeStyle>
    </UserStyle>
  </NamedLayer>
</StyledLayerDescriptor>""";
   r = requests.post(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/styles'
      ,headers={'Content-Type':'application/vnd.ogc.sld+xml'}
      ,params={'name':'catchment_polygon'}
      ,data=payload
      ,auth=('admin','nhdplus')
   );
   
if 'wbd_polygon' not in sty:
   payload = """<?xml version="1.0" encoding="UTF-8"?>
<StyledLayerDescriptor version="1.0.0" 
 xsi:schemaLocation="http://www.opengis.net/sld StyledLayerDescriptor.xsd" 
 xmlns="http://www.opengis.net/sld" 
 xmlns:ogc="http://www.opengis.net/ogc" 
 xmlns:xlink="http://www.w3.org/1999/xlink" 
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
  <NamedLayer>
    <Name>wbd_polygon</Name>
    <UserStyle>
      <FeatureTypeStyle>
        <Rule>
          <Name>Viewable</Name>
          <MaxScaleDenominator>288896</MaxScaleDenominator>
          <PolygonSymbolizer>
            <Fill>
              <CssParameter name="fill">#AAAAAA</CssParameter>
              <CssParameter name="fill-opacity">0</CssParameter>
            </Fill>
            <Stroke>
              <CssParameter name="stroke">#C500FF</CssParameter>
              <CssParameter name="stroke-width">1.5</CssParameter>
            </Stroke>
          </PolygonSymbolizer>
        </Rule>
      </FeatureTypeStyle>
    </UserStyle>
  </NamedLayer>
</StyledLayerDescriptor>""";
   r = requests.post(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/styles'
      ,headers={'Content-Type':'application/vnd.ogc.sld+xml'}
      ,params={'name':'wbd_polygon'}
      ,data=payload
      ,auth=('admin','nhdplus')
   );
   
if 'wbd2_polygon' not in sty:
   payload = """<?xml version="1.0" encoding="UTF-8"?>
<StyledLayerDescriptor version="1.0.0" 
 xsi:schemaLocation="http://www.opengis.net/sld StyledLayerDescriptor.xsd" 
 xmlns="http://www.opengis.net/sld" 
 xmlns:ogc="http://www.opengis.net/ogc" 
 xmlns:xlink="http://www.w3.org/1999/xlink" 
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
  <NamedLayer>
    <Name>wbd2_polygon</Name>
    <UserStyle>
      <FeatureTypeStyle>
        <Rule>
          <Name>Viewable</Name>
          <PolygonSymbolizer>
            <Fill>
              <CssParameter name="fill">#AAAAAA</CssParameter>
              <CssParameter name="fill-opacity">0</CssParameter>
            </Fill>
            <Stroke>
              <CssParameter name="stroke">#C500FF</CssParameter>
              <CssParameter name="stroke-width">1.5</CssParameter>
            </Stroke>
          </PolygonSymbolizer>
        </Rule>
      </FeatureTypeStyle>
    </UserStyle>
  </NamedLayer>
</StyledLayerDescriptor>""";
   r = requests.post(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/styles'
      ,headers={'Content-Type':'application/vnd.ogc.sld+xml'}
      ,params={'name':'wbd2_polygon'}
      ,data=payload
      ,auth=('admin','nhdplus')
   );
   
if 'nhdwaterbody_polygon' not in sty:
   payload = """<?xml version="1.0" encoding="UTF-8"?>
<StyledLayerDescriptor version="1.0.0" 
 xsi:schemaLocation="http://www.opengis.net/sld StyledLayerDescriptor.xsd" 
 xmlns="http://www.opengis.net/sld" 
 xmlns:ogc="http://www.opengis.net/ogc" 
 xmlns:xlink="http://www.w3.org/1999/xlink" 
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
  <NamedLayer>
    <Name>nhdwaterbody_polygon</Name>
    <UserStyle>
      <FeatureTypeStyle>
        <Rule>
          <Name>Viewable</Name>
          <MaxScaleDenominator>288896</MaxScaleDenominator>
          <PolygonSymbolizer>
            <Fill>
              <CssParameter name="fill">#97DBF2</CssParameter>
              <CssParameter name="fill-opacity">1</CssParameter>
            </Fill>
            <Stroke>
              <CssParameter name="stroke">#AAAAAA</CssParameter>
              <CssParameter name="stroke-width">0</CssParameter>
            </Stroke>
          </PolygonSymbolizer>
        </Rule>
      </FeatureTypeStyle>
    </UserStyle>
  </NamedLayer>
</StyledLayerDescriptor>""";
   r = requests.post(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/styles'
      ,headers={'Content-Type':'application/vnd.ogc.sld+xml'}
      ,params={'name':'nhdwaterbody_polygon'}
      ,data=payload
      ,auth=('admin','nhdplus')
   );

if 'nhdarea_polygon' not in sty:
   payload = """<?xml version="1.0" encoding="UTF-8"?>
<StyledLayerDescriptor version="1.0.0" 
 xsi:schemaLocation="http://www.opengis.net/sld StyledLayerDescriptor.xsd" 
 xmlns="http://www.opengis.net/sld" 
 xmlns:ogc="http://www.opengis.net/ogc" 
 xmlns:xlink="http://www.w3.org/1999/xlink" 
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
  <NamedLayer>
    <Name>nhdarea_polygon</Name>
    <UserStyle>
      <FeatureTypeStyle>
        <Rule>
          <Name>Viewable</Name>
          <MaxScaleDenominator>288896</MaxScaleDenominator>
          <PolygonSymbolizer>
            <Fill>
              <CssParameter name="fill">#70D0F8</CssParameter>
              <CssParameter name="fill-opacity">1</CssParameter>
            </Fill>
            <Stroke>
              <CssParameter name="stroke">#AAAAAA</CssParameter>
              <CssParameter name="stroke-width">0</CssParameter>
            </Stroke>
          </PolygonSymbolizer>
        </Rule>
      </FeatureTypeStyle>
    </UserStyle>
  </NamedLayer>
</StyledLayerDescriptor>""";
   r = requests.post(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/styles'
      ,headers={'Content-Type':'application/vnd.ogc.sld+xml'}
      ,params={'name':'nhdarea_polygon'}
      ,data=payload
      ,auth=('admin','nhdplus')
   );
   
if 'nhdflowline_line' not in sty:
   payload = """<?xml version="1.0" encoding="UTF-8"?>
<StyledLayerDescriptor version="1.0.0" 
 xsi:schemaLocation="http://www.opengis.net/sld StyledLayerDescriptor.xsd" 
 xmlns="http://www.opengis.net/sld" 
 xmlns:ogc="http://www.opengis.net/ogc" 
 xmlns:xlink="http://www.w3.org/1999/xlink" 
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
  <NamedLayer>
    <Name>nhdflowline_line</Name>
    <UserStyle>
      <FeatureTypeStyle>
        <Rule>
          <Name>Viewable</Name>
          <MaxScaleDenominator>288896</MaxScaleDenominator>
          <LineSymbolizer>
            <Stroke>
              <CssParameter name="stroke">#0000FF</CssParameter>
              <CssParameter name="stroke-width">1</CssParameter>
            </Stroke>
          </LineSymbolizer>
        </Rule>
      </FeatureTypeStyle>
    </UserStyle>
  </NamedLayer>
</StyledLayerDescriptor>""";
   r = requests.post(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/styles'
      ,headers={'Content-Type':'application/vnd.ogc.sld+xml'}
      ,params={'name':'nhdflowline_line'}
      ,data=payload
      ,auth=('admin','nhdplus')
   );
   
if 'nhdline_line' not in sty:
   payload = """<?xml version="1.0" encoding="UTF-8"?>
<StyledLayerDescriptor version="1.0.0" 
 xsi:schemaLocation="http://www.opengis.net/sld StyledLayerDescriptor.xsd" 
 xmlns="http://www.opengis.net/sld" 
 xmlns:ogc="http://www.opengis.net/ogc" 
 xmlns:xlink="http://www.w3.org/1999/xlink" 
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
  <NamedLayer>
    <Name>nhdline_line</Name>
    <UserStyle>
      <FeatureTypeStyle>
        <Rule>
          <Name>Viewable</Name>
          <MaxScaleDenominator>288896</MaxScaleDenominator>
          <LineSymbolizer>
            <Stroke>
              <CssParameter name="stroke">#0000FF</CssParameter>
              <CssParameter name="stroke-width">1</CssParameter>
            </Stroke>
          </LineSymbolizer>
        </Rule>
      </FeatureTypeStyle>
    </UserStyle>
  </NamedLayer>
</StyledLayerDescriptor>""";
   r = requests.post(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/styles'
      ,headers={'Content-Type':'application/vnd.ogc.sld+xml'}
      ,params={'name':'nhdline_line'}
      ,data=payload
      ,auth=('admin','nhdplus')
   );
   
if 'nhdpoint_point' not in sty:
   payload = """<?xml version="1.0" encoding="UTF-8"?>
<StyledLayerDescriptor version="1.0.0" 
 xsi:schemaLocation="http://www.opengis.net/sld StyledLayerDescriptor.xsd" 
 xmlns="http://www.opengis.net/sld" 
 xmlns:ogc="http://www.opengis.net/ogc" 
 xmlns:xlink="http://www.w3.org/1999/xlink" 
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
  <NamedLayer>
    <Name>nhdpoint_point</Name>
    <UserStyle>
      <FeatureTypeStyle>
        <Rule>
          <Name>Viewable</Name>
          <MaxScaleDenominator>288896</MaxScaleDenominator>
          <PointSymbolizer>
              <Graphic>
                <Mark>
                  <WellKnownName>square</WellKnownName>
                  <Fill>
                    <CssParameter name="fill">#0000FF</CssParameter>
                  </Fill>
                </Mark>
              <Size>6</Size>
            </Graphic>
          </PointSymbolizer>
        </Rule>
      </FeatureTypeStyle>
    </UserStyle>
  </NamedLayer>
</StyledLayerDescriptor>""";
   r = requests.post(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/styles'
      ,headers={'Content-Type':'application/vnd.ogc.sld+xml'}
      ,params={'name':'nhdpoint_point'}
      ,data=payload
      ,auth=('admin','nhdplus')
   );
   
r = requests.get(
    'http://dz_gs:8080/geoserver/rest/layers'
   ,auth=('admin','nhdplus')
);
if r.status_code != 200:
   raise Exception('layers get failed');
   
lyr = [];
if r.json()["layers"] != "":
   for item in r.json()["layers"]["layer"]:
      lyr.append(item["name"]);

if 'nhdplus:catchment_np21' not in lyr:
   payload = {
      "featureType":{
          "name":"catchment_np21"
         ,"nativeName":"catchment_np21"
         ,"namespace":{
             "name":"nhdplus"
          }
         ,"title":"catchment_np21"
         ,"nativeCRS":"EPSG:4269"
         ,"srs":"EPSG:4269"
         ,"projectionPolicy":"FORCE_DECLARED"
         ,"enabled": True
         ,"store":{
             "@class":"dataStore"
            ,"name":"nhdplus:dzpg_nhdplus"
          }
         ,"maxFeatures":0
         ,"numDecimals":0
         ,"overridingServiceSRS": False
         ,"skipNumberMatched": False
         ,"circularArcPresent": False
      }
   }
   r = requests.post(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/datastores/dzpg_nhdplus/featuretypes'
      ,headers={'Content-Type':'application/json'}
      ,data=json.dumps(payload)
      ,auth=('admin','nhdplus')
   );
   
   payload = """
<layer>
   <defaultStyle>
      <name>catchment_polygon</name>
   </defaultStyle>
</layer>"""
   r = requests.put(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/layers/catchment_np21'
      ,headers={'Content-Type':'text/xml'}
      ,data=payload
      ,auth=('admin','nhdplus')
   );
   if r.status_code != 200:
      raise Exception('layer alteration failed <' + r.status_code + '>');
   
if 'nhdplus:catchmentsp_np21' not in lyr:
   payload = {
      "featureType":{
          "name":"catchmentsp_np21"
         ,"nativeName":"catchmentsp_np21"
         ,"namespace":{
             "name":"nhdplus"
          }
         ,"title":"catchmentsp_np21"
         ,"nativeCRS":"EPSG:4269"
         ,"srs":"EPSG:4269"
         ,"projectionPolicy":"FORCE_DECLARED"
         ,"enabled": True
         ,"store":{
             "@class":"dataStore"
            ,"name":"nhdplus:dzpg_nhdplus"
          }
         ,"maxFeatures":0
         ,"numDecimals":0
         ,"overridingServiceSRS": False
         ,"skipNumberMatched": False
         ,"circularArcPresent": False
      }
   }
   r = requests.post(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/datastores/dzpg_nhdplus/featuretypes'
      ,headers={'Content-Type':'application/json'}
      ,data=json.dumps(payload)
      ,auth=('admin','nhdplus')
   );
   if r.status_code != 201:
      raise Exception('layer creation failed');
   
   payload = """
<layer>
   <defaultStyle>
      <name>catchment_polygon</name>
   </defaultStyle>
</layer>"""
   r = requests.put(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/layers/catchmentsp_np21'
      ,headers={'Content-Type':'text/xml'}
      ,data=payload
      ,auth=('admin','nhdplus')
   );
   if r.status_code != 200:
      raise Exception('layer alteration failed <' + r.status_code + '>');
      
if 'nhdplus:wbd_hu2_np21' not in lyr:
   payload = {
      "featureType":{
          "name":"wbd_hu2_np21"
         ,"nativeName":"wbd_hu2_np21"
         ,"namespace":{
             "name":"nhdplus"
          }
         ,"title":"wbd_hu2_np21"
         ,"nativeCRS":"EPSG:4269"
         ,"srs":"EPSG:4269"
         ,"projectionPolicy":"FORCE_DECLARED"
         ,"enabled": True
         ,"store":{
             "@class":"dataStore"
            ,"name":"nhdplus:dzpg_nhdplus"
          }
         ,"maxFeatures":0
         ,"numDecimals":0
         ,"overridingServiceSRS": False
         ,"skipNumberMatched": False
         ,"circularArcPresent": False
      }
   }
   r = requests.post(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/datastores/dzpg_nhdplus/featuretypes'
      ,headers={'Content-Type':'application/json'}
      ,data=json.dumps(payload)
      ,auth=('admin','nhdplus')
   );
   if r.status_code != 201:
      raise Exception('layer creation failed');
   
   payload = """
<layer>
   <defaultStyle>
      <name>wbd2_polygon</name>
   </defaultStyle>
</layer>"""
   r = requests.put(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/layers/wbd_hu2_np21'
      ,headers={'Content-Type':'text/xml'}
      ,data=payload
      ,auth=('admin','nhdplus')
   );
   if r.status_code != 200:
      raise Exception('layer alteration failed <' + r.status_code + '>');
   
if 'nhdplus:wbd_hu4_np21' not in lyr:
   payload = {
      "featureType":{
          "name":"wbd_hu4_np21"
         ,"nativeName":"wbd_hu4_np21"
         ,"namespace":{
             "name":"nhdplus"
          }
         ,"title":"wbd_hu4_np21"
         ,"nativeCRS":"EPSG:4269"
         ,"srs":"EPSG:4269"
         ,"projectionPolicy":"FORCE_DECLARED"
         ,"enabled": True
         ,"store":{
             "@class":"dataStore"
            ,"name":"nhdplus:dzpg_nhdplus"
          }
         ,"maxFeatures":0
         ,"numDecimals":0
         ,"overridingServiceSRS": False
         ,"skipNumberMatched": False
         ,"circularArcPresent": False
      }
   }
   r = requests.post(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/datastores/dzpg_nhdplus/featuretypes'
      ,headers={'Content-Type':'application/json'}
      ,data=json.dumps(payload)
      ,auth=('admin','nhdplus')
   );
   if r.status_code != 201:
      raise Exception('layer creation failed');
   
   payload = """
<layer>
   <defaultStyle>
      <name>wbd_polygon</name>
   </defaultStyle>
</layer>"""
   r = requests.put(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/layers/wbd_hu4_np21'
      ,headers={'Content-Type':'text/xml'}
      ,data=payload
      ,auth=('admin','nhdplus')
   );
   if r.status_code != 200:
      raise Exception('layer alteration failed <' + r.status_code + '>');

if 'nhdplus:wbd_hu6_np21' not in lyr:
   payload = {
      "featureType":{
          "name":"wbd_hu6_np21"
         ,"nativeName":"wbd_hu6_np21"
         ,"namespace":{
             "name":"nhdplus"
          }
         ,"title":"wbd_hu6_np21"
         ,"nativeCRS":"EPSG:4269"
         ,"srs":"EPSG:4269"
         ,"projectionPolicy":"FORCE_DECLARED"
         ,"enabled": True
         ,"store":{
             "@class":"dataStore"
            ,"name":"nhdplus:dzpg_nhdplus"
          }
         ,"maxFeatures":0
         ,"numDecimals":0
         ,"overridingServiceSRS": False
         ,"skipNumberMatched": False
         ,"circularArcPresent": False
      }
   }
   r = requests.post(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/datastores/dzpg_nhdplus/featuretypes'
      ,headers={'Content-Type':'application/json'}
      ,data=json.dumps(payload)
      ,auth=('admin','nhdplus')
   );
   if r.status_code != 201:
      raise Exception('layer creation failed');
   
   payload = """
<layer>
   <defaultStyle>
      <name>wbd_polygon</name>
   </defaultStyle>
</layer>"""
   r = requests.put(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/layers/wbd_hu6_np21'
      ,headers={'Content-Type':'text/xml'}
      ,data=payload
      ,auth=('admin','nhdplus')
   );
   if r.status_code != 200:
      raise Exception('layer alteration failed <' + r.status_code + '>');
      
if 'nhdplus:wbd_hu8_np21' not in lyr:
   payload = {
      "featureType":{
          "name":"wbd_hu8_np21"
         ,"nativeName":"wbd_hu8_np21"
         ,"namespace":{
             "name":"nhdplus"
          }
         ,"title":"wbd_hu8_np21"
         ,"nativeCRS":"EPSG:4269"
         ,"srs":"EPSG:4269"
         ,"projectionPolicy":"FORCE_DECLARED"
         ,"enabled": True
         ,"store":{
             "@class":"dataStore"
            ,"name":"nhdplus:dzpg_nhdplus"
          }
         ,"maxFeatures":0
         ,"numDecimals":0
         ,"overridingServiceSRS": False
         ,"skipNumberMatched": False
         ,"circularArcPresent": False
      }
   }
   r = requests.post(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/datastores/dzpg_nhdplus/featuretypes'
      ,headers={'Content-Type':'application/json'}
      ,data=json.dumps(payload)
      ,auth=('admin','nhdplus')
   );
   if r.status_code != 201:
      raise Exception('layer creation failed');
   
   payload = """
<layer>
   <defaultStyle>
      <name>wbd_polygon</name>
   </defaultStyle>
</layer>"""
   r = requests.put(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/layers/wbd_hu8_np21'
      ,headers={'Content-Type':'text/xml'}
      ,data=payload
      ,auth=('admin','nhdplus')
   );
   if r.status_code != 200:
      raise Exception('layer alteration failed <' + r.status_code + '>');
      
if 'nhdplus:wbd_hu10_np21' not in lyr:
   payload = {
      "featureType":{
          "name":"wbd_hu10_np21"
         ,"nativeName":"wbd_hu10_np21"
         ,"namespace":{
             "name":"nhdplus"
          }
         ,"title":"wbd_hu10_np21"
         ,"nativeCRS":"EPSG:4269"
         ,"srs":"EPSG:4269"
         ,"projectionPolicy":"FORCE_DECLARED"
         ,"enabled": True
         ,"store":{
             "@class":"dataStore"
            ,"name":"nhdplus:dzpg_nhdplus"
          }
         ,"maxFeatures":0
         ,"numDecimals":0
         ,"overridingServiceSRS": False
         ,"skipNumberMatched": False
         ,"circularArcPresent": False
      }
   }
   r = requests.post(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/datastores/dzpg_nhdplus/featuretypes'
      ,headers={'Content-Type':'application/json'}
      ,data=json.dumps(payload)
      ,auth=('admin','nhdplus')
   );
   if r.status_code != 201:
      raise Exception('layer creation failed');
   
   payload = """
<layer>
   <defaultStyle>
      <name>wbd_polygon</name>
   </defaultStyle>
</layer>"""
   r = requests.put(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/layers/wbd_hu10_np21'
      ,headers={'Content-Type':'text/xml'}
      ,data=payload
      ,auth=('admin','nhdplus')
   );
   if r.status_code != 200:
      raise Exception('layer alteration failed <' + r.status_code + '>');
      
if 'nhdplus:wbd_hu12_np21' not in lyr:
   payload = {
      "featureType":{
          "name":"wbd_hu12_np21"
         ,"nativeName":"wbd_hu12_np21"
         ,"namespace":{
             "name":"nhdplus"
          }
         ,"title":"wbd_hu12_np21"
         ,"nativeCRS":"EPSG:4269"
         ,"srs":"EPSG:4269"
         ,"projectionPolicy":"FORCE_DECLARED"
         ,"enabled": True
         ,"store":{
             "@class":"dataStore"
            ,"name":"nhdplus:dzpg_nhdplus"
          }
         ,"maxFeatures":0
         ,"numDecimals":0
         ,"overridingServiceSRS": False
         ,"skipNumberMatched": False
         ,"circularArcPresent": False
      }
   }
   r = requests.post(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/datastores/dzpg_nhdplus/featuretypes'
      ,headers={'Content-Type':'application/json'}
      ,data=json.dumps(payload)
      ,auth=('admin','nhdplus')
   );
   if r.status_code != 201:
      raise Exception('layer creation failed');
   
   payload = """
<layer>
   <defaultStyle>
      <name>wbd_polygon</name>
   </defaultStyle>
</layer>"""
   r = requests.put(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/layers/wbd_hu12_np21'
      ,headers={'Content-Type':'text/xml'}
      ,data=payload
      ,auth=('admin','nhdplus')
   );
   if r.status_code != 200:
      raise Exception('layer alteration failed <' + r.status_code + '>');
      
if 'nhdplus:nhdwaterbody_np21' not in lyr:
   payload = {
      "featureType":{
          "name":"nhdwaterbody_np21"
         ,"nativeName":"nhdwaterbody_np21"
         ,"namespace":{
             "name":"nhdplus"
          }
         ,"title":"nhdwaterbody_np21"
         ,"nativeCRS":"EPSG:4269"
         ,"srs":"EPSG:4269"
         ,"projectionPolicy":"FORCE_DECLARED"
         ,"enabled": True
         ,"store":{
             "@class":"dataStore"
            ,"name":"nhdplus:dzpg_nhdplus"
          }
         ,"maxFeatures":0
         ,"numDecimals":0
         ,"overridingServiceSRS": False
         ,"skipNumberMatched": False
         ,"circularArcPresent": False
      }
   }
   r = requests.post(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/datastores/dzpg_nhdplus/featuretypes'
      ,headers={'Content-Type':'application/json'}
      ,data=json.dumps(payload)
      ,auth=('admin','nhdplus')
   );
   if r.status_code != 201:
      raise Exception('layer creation failed');
   
   payload = """
<layer>
   <defaultStyle>
      <name>nhdwaterbody_polygon</name>
   </defaultStyle>
</layer>"""
   r = requests.put(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/layers/nhdwaterbody_np21'
      ,headers={'Content-Type':'text/xml'}
      ,data=payload
      ,auth=('admin','nhdplus')
   );
   if r.status_code != 200:
      raise Exception('layer alteration failed <' + r.status_code + '>');
      
if 'nhdplus:nhdarea_np21' not in lyr:
   payload = {
      "featureType":{
          "name":"nhdarea_np21"
         ,"nativeName":"nhdarea_np21"
         ,"namespace":{
             "name":"nhdplus"
          }
         ,"title":"nhdarea_np21"
         ,"nativeCRS":"EPSG:4269"
         ,"srs":"EPSG:4269"
         ,"projectionPolicy":"FORCE_DECLARED"
         ,"enabled": True
         ,"store":{
             "@class":"dataStore"
            ,"name":"nhdplus:dzpg_nhdplus"
          }
         ,"maxFeatures":0
         ,"numDecimals":0
         ,"overridingServiceSRS": False
         ,"skipNumberMatched": False
         ,"circularArcPresent": False
      }
   }
   r = requests.post(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/datastores/dzpg_nhdplus/featuretypes'
      ,headers={'Content-Type':'application/json'}
      ,data=json.dumps(payload)
      ,auth=('admin','nhdplus')
   );
   if r.status_code != 201:
      raise Exception('layer creation failed');
   
   payload = """
<layer>
   <defaultStyle>
      <name>nhdarea_polygon</name>
   </defaultStyle>
</layer>"""
   r = requests.put(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/layers/nhdarea_np21'
      ,headers={'Content-Type':'text/xml'}
      ,data=payload
      ,auth=('admin','nhdplus')
   );
   if r.status_code != 200:
      raise Exception('layer alteration failed <' + r.status_code + '>');
      
if 'nhdplus:nhdflowline_np21' not in lyr:
   payload = {
      "featureType":{
          "name":"nhdflowline_np21"
         ,"nativeName":"nhdflowline_np21"
         ,"namespace":{
             "name":"nhdplus"
          }
         ,"title":"nhdflowline_np21"
         ,"nativeCRS":"EPSG:4269"
         ,"srs":"EPSG:4269"
         ,"projectionPolicy":"FORCE_DECLARED"
         ,"enabled": True
         ,"store":{
             "@class":"dataStore"
            ,"name":"nhdplus:dzpg_nhdplus"
          }
         ,"maxFeatures":0
         ,"numDecimals":0
         ,"overridingServiceSRS": False
         ,"skipNumberMatched": False
         ,"circularArcPresent": False
      }
   }
   r = requests.post(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/datastores/dzpg_nhdplus/featuretypes'
      ,headers={'Content-Type':'application/json'}
      ,data=json.dumps(payload)
      ,auth=('admin','nhdplus')
   );
   if r.status_code != 201:
      raise Exception('layer creation failed');
   
   payload = """
<layer>
   <defaultStyle>
      <name>nhdflowline_line</name>
   </defaultStyle>
</layer>"""
   r = requests.put(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/layers/nhdflowline_np21'
      ,headers={'Content-Type':'text/xml'}
      ,data=payload
      ,auth=('admin','nhdplus')
   );
   if r.status_code != 200:
      raise Exception('layer alteration failed <' + r.status_code + '>');
      
if 'nhdplus:nhdline_np21' not in lyr:
   payload = {
      "featureType":{
          "name":"nhdline_np21"
         ,"nativeName":"nhdline_np21"
         ,"namespace":{
             "name":"nhdplus"
          }
         ,"title":"nhdline_np21"
         ,"nativeCRS":"EPSG:4269"
         ,"srs":"EPSG:4269"
         ,"projectionPolicy":"FORCE_DECLARED"
         ,"enabled": True
         ,"store":{
             "@class":"dataStore"
            ,"name":"nhdplus:dzpg_nhdplus"
          }
         ,"maxFeatures":0
         ,"numDecimals":0
         ,"overridingServiceSRS": False
         ,"skipNumberMatched": False
         ,"circularArcPresent": False
      }
   }
   r = requests.post(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/datastores/dzpg_nhdplus/featuretypes'
      ,headers={'Content-Type':'application/json'}
      ,data=json.dumps(payload)
      ,auth=('admin','nhdplus')
   );
   if r.status_code != 201:
      raise Exception('layer creation failed');
   
   payload = """
<layer>
   <defaultStyle>
      <name>nhdline_line</name>
   </defaultStyle>
</layer>"""
   r = requests.put(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/layers/nhdline_np21'
      ,headers={'Content-Type':'text/xml'}
      ,data=payload
      ,auth=('admin','nhdplus')
   );
   if r.status_code != 200:
      raise Exception('layer alteration failed <' + r.status_code + '>');
      
if 'nhdplus:nhdpoint_np21' not in lyr:
   payload = {
      "featureType":{
          "name":"nhdpoint_np21"
         ,"nativeName":"nhdpoint_np21"
         ,"namespace":{
             "name":"nhdplus"
          }
         ,"title":"nhdpoint_np21"
         ,"nativeCRS":"EPSG:4269"
         ,"srs":"EPSG:4269"
         ,"projectionPolicy":"FORCE_DECLARED"
         ,"enabled": True
         ,"store":{
             "@class":"dataStore"
            ,"name":"nhdplus:dzpg_nhdplus"
          }
         ,"maxFeatures":0
         ,"numDecimals":0
         ,"overridingServiceSRS": False
         ,"skipNumberMatched": False
         ,"circularArcPresent": False
      }
   }
   r = requests.post(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/datastores/dzpg_nhdplus/featuretypes'
      ,headers={'Content-Type':'application/json'}
      ,data=json.dumps(payload)
      ,auth=('admin','nhdplus')
   );
   if r.status_code != 201:
      raise Exception('layer creation failed');
   
   payload = """
<layer>
   <defaultStyle>
      <name>nhdpoint_point</name>
   </defaultStyle>
</layer>"""
   r = requests.put(
       'http://dz_gs:8080/geoserver/rest/workspaces/nhdplus/layers/nhdpoint_np21'
      ,headers={'Content-Type':'text/xml'}
      ,data=payload
      ,auth=('admin','nhdplus')
   );
   if r.status_code != 200:
      raise Exception('layer alteration failed <' + r.status_code + '>');

